In [1]:
import json
import pandas as pd
from tqdm import tqdm
from langchain_community.llms import Ollama

# Dataset

In [4]:
def read_json_array(path):
  data = []
  with open(path, 'r') as file:
    for line in file:
      json_object = json.loads(line.strip())
      data.append(json_object)
  return data

dataset_path = '../../datasets/amazon_reviews/'
data_path = dataset_path + "Musical_Instruments.json"

data = read_json_array(data_path)
df = pd.DataFrame(data)

In [5]:
amazon_reviews = pd.DataFrame()
amazon_reviews['Reviews'] = df['reviewText'].copy()
amazon_reviews['Ratings'] = df['overall'].copy()
amazon_reviews = amazon_reviews.sample(n=10000, random_state=42).reset_index(drop=True)

# Modeling

In [6]:
llm = Ollama(model="gemma2")

/var/folders/87/dhv__9sj0yv1kz0pfv2ds9nh0000gn/T/ipykernel_1168/2721138275.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma2")


### Step 1 - Creating Topics for batches of dataset

In [37]:
def generate_topics_in_batches(amazon_reviews, batch_size=100, llm=None):
    """
    Generate topics from Amazon reviews in batches to handle large datasets.
    
    Parameters:
    amazon_reviews (pd.DataFrame): DataFrame containing Amazon reviews
    batch_size (int): Number of reviews to process in each batch
    llm: The language model instance to use for generating topics
    
    Returns:
    list: List of lists containing topics for each batch
    """
    if llm is None:
        raise ValueError("LLM instance must be provided")
    
    all_batch_topics = []
    total_reviews = len(amazon_reviews)
    
    # Process reviews in batches
    for start_idx in tqdm(range(0, total_reviews, batch_size)):
        end_idx = min(start_idx + batch_size, total_reviews)
        batch_reviews = amazon_reviews.iloc[start_idx:end_idx]
        
        # Generate prompt for current batch
        prompt_labels_generator = f'''You are provided with amazon reviews and helping to cluster the reviews based on the topics.
Please create topics based on the reviews provided. Keep the topics general and not specific to the reviews. 
DO NOT include any specific details. NO TOPICS LIKE  DJ Equipment, Guitar, Piano, etc. Topics should be general and helpful for business steakholders to understand what people are thinking about the products,
Amazon Reviews: {batch_reviews['Reviews'].tolist()}
Please return only in CSV format with the following structure:
Topic1, Topic2, Topic3, Topic4, Topic5,...
Return only the topics in CSV format and nothing else.
'''

        # Get topics for current batch
        result = llm.invoke(prompt_labels_generator, temperature=0.0)
        
        # Convert CSV string to list of topics
        batch_topics = [topic.strip() for topic in result.split(',')]
        all_batch_topics.append(batch_topics)
        
        print(f"Processed batch {len(all_batch_topics)}: reviews {start_idx} to {end_idx}")
    
    return all_batch_topics

In [38]:
topics_by_batch = generate_topics_in_batches(amazon_reviews.sample(1000, random_state=42), batch_size=10, llm=llm)

  1%|          | 1/100 [00:04<08:06,  4.92s/it]

Processed batch 1: reviews 0 to 10


  2%|▏         | 2/100 [00:09<07:51,  4.81s/it]

Processed batch 2: reviews 10 to 20


  3%|▎         | 3/100 [00:13<07:23,  4.57s/it]

...

Processed batch 98: reviews 970 to 980


 99%|█████████▉| 99/100 [06:57<00:03,  3.62s/it]

Processed batch 99: reviews 980 to 990


100%|██████████| 100/100 [07:01<00:00,  4.21s/it]

Processed batch 100: reviews 990 to 1000


In [39]:
all_unique_topics = list(set([topic for batch in topics_by_batch for topic in batch]))

In [41]:
all_unique_topics

['Technical Aspects',
 'User Experience',
 'Sound & Performance',
 'Guitar Straps',
 'Sound/Performance',
 'Aesthetics',
 'Guitar Effects Pedals',
 'Impedance Matching',
 'Customer Experience',
 'Product Durability',
 'Performance & Functionality',
 'Price Value',
 'Durability & Performance',
 'Features',
 'Value',
 'Comfort',
 'Shipping & Delivery',
 'Microphone Sensitivity',
 'Quality',
 'Learning Curve',
 'Size',
 'Usage Experience',
 'Vocal Harmonizers',
 'Product Quality',
 'Size & Portability',
 'Value for Money',
 'Price',
 'Performance Issues',
 'Guitar Amplifiers',
 'Usage',
 'Aesthetics & Comfort',
 'Performance',
 'Delivery Experience',
 'Customer Satisfaction',
 'Installation & Setup',
 'Target Audience',
 'Comfort/Design',
 'Fit & Functionality',
 'Brand Perception',
 'Experience',
 'Cello Pickup Installation',
 'Shipping Experience',
 'Compatibility',
 'Customer Service',
 'Product Performance',
 'Usability',
 'Quality & Durability',
 'Performance Quality',
 'Shipping & P

### Step 2 - Combining all topics to a smaller more general subset

In [40]:
prompt_merge_topics = f'''You are provided with topics generated from Amazon reviews.
Please merge the topics into a smaller number of topics. The topics should be VERY GENERAL and NOT specific. Do not output more than 15 topics. 
Merge/combine topics that are similar or related.
Topics: {all_unique_topics}
Please return only in CSV format with the following structure:
MergedTopic1, MergedTopic2, MergedTopic3, MergedTopic4, MergedTopic5,...
Return only the merged topics in CSV format and nothing else.
'''

# Get merged topics
proper_topics_str = llm.invoke(prompt_merge_topics, temperature=0.0)

In [43]:
proper_topics = [topic.strip() for topic in proper_topics_str.split(',')]
print(proper_topics)

['Technical Aspects', 'User Experience', 'Sound & Performance', 'Aesthetics', 'Durability', 'Price & Value', 'Customer Experience', 'Size & Portability', 'Functionality & Usability', 'Shipping & Delivery', 'Brand Perception', 'Product Quality', 'Comfort & Design', 'Learning Curve', 'Target Audience', 'Usage Experience', 'Gift Giving', 'Emotional Response', 'Compatibility', 'Customer Service']


### Step 3 - Assigning batches of dataset to Topics Generated in Step 2 and sentiment of the review

In [44]:
def assign_topics_in_batches(input_df, topics, batch_size, llm=None):
    """
    Assign topics and sentiment to Amazon reviews in batches and update DataFrame directly.

    Parameters:
    input_df (pd.DataFrame): DataFrame containing Amazon reviews
    topics (str): String of topics to assign from
    batch_size (int): Number of reviews to process in each batch
    llm: The language model instance to use for assigning topics

    Returns:
    pd.DataFrame: Updated DataFrame with topic and sentiment assignments
    """
    if llm is None:
        raise ValueError("LLM instance must be provided")

    # Create a copy of the DataFrame to avoid modifying the original
    df = input_df.copy()

    # Initialize Topic and Sentiment columns with 'Unknown'
    df['Topic'] = 'Unknown'
    df['Sentiment'] = 'Unknown'

    total_reviews = len(df)

    # Process reviews in batches with progress bar
    for start_idx in tqdm(range(0, total_reviews, batch_size), desc="Assigning topics"):
        end_idx = min(start_idx + batch_size, total_reviews)
        batch_reviews_list = df.iloc[start_idx:end_idx]
        batch_reviews = " ".join([f"Comment {i + 1}: {review}," for i, review in enumerate(batch_reviews_list['Reviews'])])
        
        # Generate prompt for current batch
        prompt_assigning_prompt = f'''You are provided with amazon reviews and helping to cluster the reviews based on the topics.
Please assign the review to the topics provided. Return only the name of the topic and sentiment for the review. Sentiment can be only Positive, Negative or Neutral.
Amazon Review: {batch_reviews}
Topics: {topics}
Please return in JSON format only topics and sentiment for respective reviews and nothing else. Do not use triple backtick blocks. Only output exactly as on the example below:
Example: Having an input of Review1, Review2, Review3, Review4
Output: [{{"topic": "Topic1", "sentiment": "Sentiment1"}}, {{"topic": "Topic2", "sentiment": "Sentiment2"}}, {{"topic": "Topic3", "sentiment": "Sentiment3"}}]
'''

        # Get assignments for current batch
        result = llm.invoke(prompt_assigning_prompt, temperature=0.0)

        try:
            # Parse JSON response
            assignments = json.loads(result)

            # Update DataFrame directly using indices
            for idx, assignment in enumerate(assignments):
                current_idx = start_idx + idx
                if current_idx < total_reviews:
                    topic = assignment.get('topic', 'Unknown')
                    sentiment = assignment.get('sentiment', 'Unknown')

                    df.at[current_idx, 'Topic'] = topic
                    df.at[current_idx, 'Sentiment'] = sentiment

        except json.JSONDecodeError as e:
            print(f"\nError parsing JSON for batch starting at index {start_idx}: {str(e)}")
            print(f"Raw response: {result}")
            continue
        except Exception as e:
            print(f"\nUnexpected error processing batch starting at index {start_idx}: {str(e)}")
            continue

    return df


In [45]:
amazon_reviews_with_topics = assign_topics_in_batches(
    amazon_reviews,
    topics=str(proper_topics),
    batch_size=5,
    llm=llm
)

Assigning topics:   0%|          | 1/2000 [00:06<3:22:40,  6.08s/it]


Error parsing JSON for batch starting at index 0: Expecting value: line 1 column 299 (char 298)
Raw response: [{"topic": "User Experience", "sentiment": "Negative"}, {"topic": "Product Quality", "sentiment": "Positive"}, {"topic": "Sound & Performance", "sentiment": "Positive"}, {"topic": "Functionality & Usability", "sentiment": "Positive"}, {"topic": "Functionality & Usability", "sentiment": "Neutral"},] 



Assigning topics:   1%|          | 23/2000 [02:11<3:00:02,  5.46s/it]


Error parsing JSON for batch starting at index 110: Expecting value: line 1 column 284 (char 283)
Raw response: [{"topic": "Durability", "sentiment": "Positive"}, {"topic": "Functionality & Usability", "sentiment": "Positive"}, {"topic": "Product Quality", "sentiment": "Positive"}, {"topic": "Learning Curve", "sentiment": "Positive"}, {"topic": "Sound & Performance", "sentiment": "Positive"},] 



Assigning topics:   7%|▋         | 135/2000 [13:54<2:57:49,  5.72s/it]

...


Error parsing JSON for batch starting at index 6405: Expecting value: line 1 column 292 (char 291)
Raw response: [{"topic": "User Experience", "sentiment": "Positive"}, {"topic": "Price & Value", "sentiment": "Positive"}, {"topic": "Sound & Performance", "sentiment": "Positive"}, {"topic": "Functionality & Usability", "sentiment": "Positive"}, {"topic": "Customer Experience", "sentiment": "Negative"},] 



Assigning topics:  69%|██████▉   | 1387/2000 [2:24:55<1:03:04,  6.17s/it]


Error parsing JSON for batch starting at index 6930: Expecting value: line 1 column 271 (char 270)
Raw response: [{"topic": "Technical Aspects", "sentiment": "Positive"}, {"topic": "Product Quality", "sentiment": "Negative"}, {"topic": "Durability", "sentiment": "Negative"}, {"topic": "Sound & Performance", "sentiment": "Negative"}, {"topic": "Aesthetics", "sentiment": "Neutral"},] 



Assigning topics:  73%|███████▎  | 1463/2000 [2:33:05<56:11,  6.28s/it]  


KeyboardInterrupt: 

In [46]:
amazon_reviews_with_topics

NameError: name 'amazon_reviews_with_topics' is not defined

In [14]:
amazon_reviews_with_topics.to_csv('outputs/amazon_reviews_with_topics_10000.csv', index=False)

In [15]:
amazon_reviews_with_topics['Topic'].value_counts()

Topic
Learning Experience    44
User Satisfaction      19
Product Quality         6
Unknown                 1
Name: count, dtype: int64

In [19]:
amazon_reviews_with_topics.to_excel('outputs/amazon_reviews_with_topics.xlsx', index=False)

In [20]:
amazon_reviews_with_topics['Sentiment'].value_counts()

Sentiment
Positive    58
Negative     7
Neutral      4
Unknown      1
Name: count, dtype: int64

In [21]:
amazon_reviews_with_topics[amazon_reviews_with_topics['Sentiment'] != 'Positive']


,Reviews,Ratings,Topic,Sentiment
11,Not really current.,3.0,Product Quality,Negative
14,Good reference book,5.0,Product Quality,Neutral
17,I'm only giving this book three stars because ...,3.0,Learning Experience,Negative
23,still can't crochet to save my life,4.0,Learning Experience,Negative
26,:-),5.0,Learning Experience,Neutral
45,Great book I just wish the pics were more in c...,5.0,Product Quality,Negative
47,I TRIED BUT I AM BETTER AT KNITTING,5.0,Learning Experience,Negative
52,still learning but having fun doing it,5.0,Learning Experience,Neutral
62,Thanks,5.0,User Satisfaction,Neutral
67,I have never really used it. I don't really ge...,2.0,User Satisfaction,Negative
